In [1]:
#setup - rememeber to switch to tensorflow 2.3 kernel...
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io as sio
import datetime
import trimesh
import time
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget

#need to have these two lines to work on my ancient 1060 3gb
#  https://stackoverflow.com/questions/43990046/tensorflow-blas-gemm-launch-failed
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print(tf.__version__)

# %matplotlib inline
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'
%matplotlib notebook

%load_ext tensorboard

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180

C:\Users\Derm\anaconda3\envs\tf23\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Derm\anaconda3\envs\tf23\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Derm\anaconda3\envs\tf23\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


2.3.0


Autosaving every 180 seconds


In [2]:
#load mesh of object
surf_fn = "C:/Users/Derm/vaRLnt/v3/demo/sigmaPoint/wall_scaled.stl"
M = trimesh.load(surf_fn)
#get ground truth translation
gt = np.loadtxt("C:/Users/Derm/vaRLnt/v3/demo/sigmaPoint/gt.txt")
print(gt)
#load point clouds of surface generated in Matlab
pts1_fn = "C:/Users/Derm/vaRLnt/v3/demo/sigmaPoint/s1.txt"
pts1 = np.loadtxt(pts1_fn)
pts2_fn = "C:/Users/Derm/vaRLnt/v3/demo/sigmaPoint/s2.txt"
pts2 = np.loadtxt(pts2_fn)
# pts = pts - gt #+ np.array([20,5,1.72])

[-40.   -40.    -1.72]


In [3]:
#plot with Vedo
plt1 = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp = []
disp.append(Points(pts1, c = 'blue', r = 5))
w = Mesh(M).c("gray").alpha(1)
w.pos(gt[0], gt[1], gt[2])
disp.append(w)
# plt1.show(disp, "Fig 2.1")
# ViewInteractiveWidget(plt1.window)

## Plot Points + distributions

In [14]:
#Run ICET
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(os.path.dirname(currentdir))
sys.path.insert(0, parentdir)
from ICET_spherical import ICET
# print(currentdir)
# print(parentdir)

it1 = ICET(cloud1 = pts1, cloud2 = pts2,  fid = 40, draw = True, 
            niter = 5, group= 2, RM = False, DNN_filter = False)
ViewInteractiveWidget(it1.plt.window)


 jumps 
 tf.Tensor(
[[ 102    4]
 [ 116    5]
 [ 945    0]
 [1257    3]
 [1271    2]], shape=(5, 2), dtype=int64)
used tf.Tensor([[4 5 0 3 2]], shape=(1, 5), dtype=int64)
missing tf.Tensor([[1]], shape=(1, 1), dtype=int64)
z tf.Tensor([[0]], shape=(1, 1), dtype=int64)
OG_rads tf.Tensor(
[[62.233017 55.3377   51.28682  51.299477 52.1207   52.549694]
 [55.97538  59.062405 50.991444 52.13219  52.109894 52.91746 ]
 [54.48706  54.104618 50.72865  51.717876 52.075397 52.674427]
 ...
 [ 0.       53.510048  0.        0.        0.        0.      ]
 [ 0.       54.902714  0.        0.        0.        0.      ]
 [ 0.       62.251385  0.        0.        0.        0.      ]], shape=(1304, 6), dtype=float32)
tf.Tensor([[55.97538  59.062405 50.991444 52.13219  52.109894 52.91746 ]], shape=(1, 6), dtype=float32)
tf.Tensor([[0 1]], shape=(1, 2), dtype=int64)

 jumps after fix tf.Tensor(
[[ 102    4]
 [ 116    5]
 [ 945    0]
 [1257    3]
 [1271    2]
 [   0    1]], shape=(6, 2), dtype=int64)
jumps_i 

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [ ]:
#unit test for fixing no skip bug
all_spikes = tf.cast(tf.linspace(0,4,5), tf.int64)[None,:]
used = tf.constant([0, 1, 5], dtype = tf.int64)[None,:]
print(all_spikes)
print(used)

to_add = tf.sets.difference(all_spikes, used).values[None,:]
z = tf.zeros(tf.shape(to_add), dtype = tf.int64)
# print(z)
to_add = tf.transpose(tf.concat((z, to_add), axis = 0))
print(to_add)

# print(tf.math.reduce_max(used))

t = tf.constant([[1,1]], dtype = tf.int64)
print(t)
ans = tf.concat((t, to_add), axis = 0)
print(ans)
